# Gradio and Huggingface Tutorial

This tutorial covers the basics of building interactive machine learning applications using Gradio, with a focus on Huggingface model integration and creating organized layouts.

## Installation

First, we need to install the required libraries. Gradio is our interface builder, and transformers gives us access to Huggingface models.

In [ ]:
# Install the necessary packages
# This may take a minute or two depending on your internet connection
!pip install gradio transformers torch pillow

## Part 1: Basic Gradio Interface

Let's start with a simple example to understand how Gradio works. We'll create a basic text processing function.

In [ ]:
import gradio as gr

# Define a simple function that we want to expose through a web interface
# This function takes text input and returns it in uppercase
def convert_to_uppercase(text):
    # Process the input text
    result = text.upper()
    return result

# Create the Gradio interface
# The Interface class connects our function to a web UI
demo = gr.Interface(
    fn=convert_to_uppercase,           # The function to call
    inputs="text",                      # Type of input component
    outputs="text",                     # Type of output component
    title="Simple Text Processor",      # Title shown at the top
    description="Enter text to convert it to uppercase"  # Help text for users
)

# Launch the interface
# This creates a local web server and opens the app in your browser
demo.launch()

## Part 2: Working with Images

Gradio makes it easy to work with images. Let's create an app that applies basic image transformations.

In [ ]:
import gradio as gr
from PIL import Image, ImageFilter

# Function to apply a blur filter to an image
# Gradio automatically handles image upload and display
def blur_image(image):
    # The image comes in as a PIL Image object
    # Apply a Gaussian blur filter
    blurred = image.filter(ImageFilter.GaussianBlur(radius=5))
    return blurred

# Create an interface for image processing
demo = gr.Interface(
    fn=blur_image,
    inputs=gr.Image(type="pil"),        # Input is an image, returned as PIL format
    outputs=gr.Image(type="pil"),       # Output is also an image
    title="Image Blur Tool",
    description="Upload an image to apply a blur effect"
)

demo.launch()

## Part 3: Using Rows and Columns for Layout

Gradio provides Blocks, which allows us to create custom layouts using rows and columns. This gives us much more control over the interface design.

In [ ]:
import gradio as gr

# Define multiple functions for our multi-feature app
def greet_user(name):
    return f"Hello, {name}! Welcome to the app."

def calculate_square(number):
    result = number ** 2
    return result

# Create a Blocks interface for custom layout
# Blocks gives us fine-grained control over component placement
with gr.Blocks() as demo:
    # Add a title at the top
    gr.Markdown("# Multi-Function Application")
    gr.Markdown("This app demonstrates using rows and columns in Gradio")
    
    # Create a row to place components side by side
    with gr.Row():
        # First column - greeting function
        with gr.Column():
            gr.Markdown("### Greeting Section")
            name_input = gr.Textbox(label="Your Name", placeholder="Enter your name")
            greet_button = gr.Button("Greet Me")
            greet_output = gr.Textbox(label="Greeting")
        
        # Second column - calculator function
        with gr.Column():
            gr.Markdown("### Calculator Section")
            number_input = gr.Number(label="Enter a Number")
            calc_button = gr.Button("Calculate Square")
            calc_output = gr.Number(label="Result")
    
    # Connect the buttons to their respective functions
    # The click event triggers when the button is pressed
    greet_button.click(
        fn=greet_user,
        inputs=name_input,
        outputs=greet_output
    )
    
    calc_button.click(
        fn=calculate_square,
        inputs=number_input,
        outputs=calc_output
    )

demo.launch()

## Part 4: Integrating Huggingface Models

Now let's integrate a real machine learning model from Huggingface. We'll use a sentiment analysis model as an example.

In [ ]:
import gradio as gr
from transformers import pipeline

# Load a pre-trained sentiment analysis model from Huggingface
# The pipeline API makes it easy to use models without detailed configuration
# This will download the model on first run, which may take a moment
sentiment_analyzer = pipeline("sentiment-analysis")

# Define a function that uses the Huggingface model
def analyze_sentiment(text):
    # Pass the text to the model for analysis
    result = sentiment_analyzer(text)[0]
    
    # Extract the predicted label and confidence score
    label = result['label']
    score = result['score']
    
    # Format the output in a readable way
    output = f"Sentiment: {label}\nConfidence: {score:.2%}"
    return output

# Create the interface
demo = gr.Interface(
    fn=analyze_sentiment,
    inputs=gr.Textbox(lines=3, placeholder="Enter text to analyze..."),
    outputs=gr.Textbox(label="Analysis Result"),
    title="Sentiment Analysis with Huggingface",
    description="This app uses a pre-trained model to determine if text is positive or negative",
    examples=[
        ["I love this product! It's amazing!"],
        ["This is terrible and disappointing."]
    ]
)

demo.launch()

## Part 5: Advanced Layout with Huggingface Image Model

Let's create a more complex application that combines everything we've learned: rows, columns, images, and Huggingface models.

In [ ]:
import gradio as gr
from transformers import pipeline

# Load an image classification model from Huggingface
# This model can identify objects in images
image_classifier = pipeline("image-classification")

# Function to classify images using the Huggingface model
def classify_image(image):
    # Get predictions from the model
    # The model returns the top predictions with confidence scores
    predictions = image_classifier(image)
    
    # Format the results as a dictionary for Gradio to display
    # This will create a nice label-confidence visualization
    results = {pred['label']: pred['score'] for pred in predictions}
    return results

# Create a comprehensive interface with custom layout
with gr.Blocks() as demo:
    # Header section
    gr.Markdown("# Image Classification with Huggingface")
    gr.Markdown("Upload an image to see what objects the AI model can identify")
    
    # Main content in a row
    with gr.Row():
        # Left column for input
        with gr.Column():
            gr.Markdown("### Input Image")
            image_input = gr.Image(type="pil")
            classify_button = gr.Button("Classify Image", variant="primary")
        
        # Right column for output
        with gr.Column():
            gr.Markdown("### Classification Results")
            # Label component shows confidence scores in a clean format
            classification_output = gr.Label(num_top_classes=5)
    
    # Example images row at the bottom
    gr.Markdown("### Try These Example Images")
    gr.Examples(
        examples=[
            # You can add paths to example images here
            # For this demo, users will need to upload their own images
        ],
        inputs=image_input
    )
    
    # Connect the button to the classification function
    classify_button.click(
        fn=classify_image,
        inputs=image_input,
        outputs=classification_output
    )

demo.launch()

## Part 6: Complete Application with Multiple Models

Here's a final example that combines text and image processing in one application.

In [ ]:
import gradio as gr
from transformers import pipeline

# Initialize both models
# We're creating separate pipelines for text and image tasks
text_model = pipeline("sentiment-analysis")
image_model = pipeline("image-classification")

# Text analysis function
def analyze_text(text):
    result = text_model(text)[0]
    return f"{result['label']} (Confidence: {result['score']:.2%})"

# Image analysis function
def analyze_image(image):
    predictions = image_model(image)
    return {pred['label']: pred['score'] for pred in predictions}

# Build a professional-looking multi-model application
with gr.Blocks(title="AI Analysis Suite") as demo:
    # Main header
    gr.Markdown("# AI Analysis Suite")
    gr.Markdown("Choose between text sentiment analysis or image classification")
    
    # Create tabs for different functionalities
    # Tabs are another way to organize content besides rows and columns
    with gr.Tab("Text Analysis"):
        with gr.Row():
            with gr.Column():
                text_input = gr.Textbox(
                    lines=5,
                    label="Input Text",
                    placeholder="Enter your text here..."
                )
                text_button = gr.Button("Analyze Sentiment")
            
            with gr.Column():
                text_output = gr.Textbox(label="Sentiment Result")
        
        text_button.click(
            fn=analyze_text,
            inputs=text_input,
            outputs=text_output
        )
    
    with gr.Tab("Image Classification"):
        with gr.Row():
            with gr.Column():
                image_input = gr.Image(type="pil", label="Upload Image")
                image_button = gr.Button("Classify Image")
            
            with gr.Column():
                image_output = gr.Label(num_top_classes=5, label="Top Predictions")
        
        image_button.click(
            fn=analyze_image,
            inputs=image_input,
            outputs=image_output
        )
    
    # Footer with information
    gr.Markdown("---")
    gr.Markdown("Built with Gradio and Huggingface Transformers")

# Launch the application
# Setting share=False keeps it local, set to True to get a public link
demo.launch(share=False)

## Summary

### What We Covered

1. **Basic Gradio Interfaces**: Simple function wrapping with Interface class
2. **Image Handling**: Working with image inputs and outputs
3. **Custom Layouts**: Using Blocks, Rows, and Columns for organized interfaces
4. **Huggingface Integration**: Loading and using pre-trained models
5. **Advanced Applications**: Combining multiple models and layouts

### Key Concepts

- **Interface**: Quick way to create a web app from a Python function
- **Blocks**: Advanced layout system for custom designs
- **Row**: Places components horizontally side by side
- **Column**: Stacks components vertically within a row
- **Pipeline**: Huggingface's simple API for using pre-trained models

### Next Steps

- Experiment with different Huggingface models from the model hub
- Try combining multiple rows and columns for complex layouts
- Add custom CSS styling to your Gradio apps
- Deploy your app using Huggingface Spaces or other hosting platforms

Gradio makes it incredibly easy to turn your machine learning projects into interactive web applications that anyone can use, without requiring any web development knowledge.